In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

root = '3-condition-splited-data'

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),         
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = ImageFolder(root=root + '/train', transform=transform)
test_datas = ImageFolder(root=root + '/test', transform=transform)
val_datas = ImageFolder(root=root+"/validation",transform=transform)

batch_size = 16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_datas, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_datas, batch_size=batch_size, shuffle=False)

C:\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
C:\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
import torch
import torch.nn as nn

class ANN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, hidden_size4, num_classes):
        super(ANN, self).__init__()
        self.flatten = nn.Flatten()
        
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.batch1 = nn.BatchNorm1d(hidden_size1)
        self.drop1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.batch2 = nn.BatchNorm1d(hidden_size2)
        self.drop2 = nn.Dropout(0.5)

        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.relu3 = nn.ReLU()
        self.batch3 = nn.BatchNorm1d(hidden_size3)
        self.drop3 = nn.Dropout(0.5)

        self.fc4 = nn.Linear(hidden_size3, hidden_size4)
        self.relu4 = nn.ReLU()
        self.batch4 = nn.BatchNorm1d(hidden_size4)
        self.drop4 = nn.Dropout(0.5)

        self.fc5 = nn.Linear(hidden_size4, num_classes)

    def forward(self, x):
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.batch1(x)
        x = self.drop1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.batch2(x)
        x = self.drop2(x)

        x = self.fc3(x)
        x = self.relu3(x)
        x = self.batch3(x)
        x = self.drop3(x)

        x = self.fc4(x)
        x = self.relu4(x)
        x = self.batch4(x)
        x = self.drop4(x)

        x = self.fc5(x)

        return x


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
INPUT_SIZE = (256,256,3)

W,H,C = INPUT_SIZE
input_size = W*H*C
model = ANN(input_size,256,128,64,32,len(train_data.classes)).to(device)

saved_weights_path = "ann-3-fit.pth"

model.load_state_dict(torch.load(saved_weights_path))

<All keys matched successfully>

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()

model.eval()  # Set the model to evaluation mode
test_loss = 0.0
correct = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        
        
        
        all_labels += labels.cpu().numpy().tolist()
        all_predictions += predicted.cpu().numpy().tolist()

test_loss /= len(test_loader.dataset)
accuracy = correct / len(test_loader.dataset) * 100.0

print(f'Test Loss: {test_loss}, Test Accuracy: {accuracy}%')


Test Loss: 0.0007457558874108751, Test Accuracy: 99.6%


In [5]:
# Accuracy,  Precision, Recall ve F1 skoru, Karmaşıklık Matrisi, Classification Report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')
f1 = f1_score(all_labels, all_predictions, average='weighted')
cm = confusion_matrix(all_labels, all_predictions)

print(f'Test Loss: {test_loss:.4f}')
print(f"Test Accuracy : {accuracy:.4f}")
print(f'Precision: {precision:.4f}')
print(f"Recall : {recall:.4f}")
print(f"F1 Score : {f1:.4f}")
print('Confusion Matrix:')
print(cm)
print('\nClassification Report:')
print(classification_report(all_labels, all_predictions, target_names=[str(i) for i in range(len(test_datas.classes))]))


Test Loss: 0.0007
Test Accuracy : 99.6000
Precision: 0.9960
Recall : 0.9960
F1 Score : 0.9960
Confusion Matrix:
[[1500    0]
 [  12 1488]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      1500
           1       1.00      0.99      1.00      1500

    accuracy                           1.00      3000
   macro avg       1.00      1.00      1.00      3000
weighted avg       1.00      1.00      1.00      3000

